In [1]:
import numpy as np
from scripts import *

In [2]:
# Calibration of the reference mantle density with temperature dependent only densities (model M2)
#
# STEP 1: Definition of a semi-analytical MOR (MOR2)
#
# STEP 2: Calibration of the reference mantle density using MOR2
#

In [3]:
# Calibration of the reference mantle density with temperature dependent only densities (model M2)
#
# STEP 1: Definition of a semi-analytical MOR 

water_depth_at_ridge     = 2750.0e0  # meters = reference water depth at the ridge based on average MOR elevation
sea_level_ref            = -400e0    # meters = reference sea level based on average elevation of continents
dz                       = 200e0     # meters
ref_depth                = 125e3     # meters = compensation depth (base of the continental lithosphere)

# 1) Definition of a semi-anlytical MOR that fits the relative elevation between MOR and continents obtained in M2


# Lithosphere 1 = Reference continental column
reflithcont             = 'lith125'
book_reflithcont        = book_lith125_GL # updated parameters (./scripts/book_lith.py)

# Reference MOR column (geotherm and densities from model M2)
# The MOR from 2-D thermechanical model is used to calibrate density structure of the reference MOR2 column
reflith                 = 'MOR'
book_reflith            = book_ridge

# Lithosphere 2 = Reference MOR2 column with semi-analytical geotherm and density structure
# 
complith                 = 'MOR2'
book_complith            = book_ridge

printOut                 = False          # print more information

# Average depletion at the MOR2 is -12 kg/m3 between 6.5 and 125 km depth similarly to model M2
# We look for a correction in order to fit the relative elevation between MOR and continents obtained in M2
#alldeltarho_ridge_correction_MOR2 = np.arange(-11.3,-10.8,0.1)
alldeltarho_ridge_correction_MOR2 = [ -11.25 ]  # solution where dh ~ 0

book_reflith  = update_mantle_density_to_reference(reflith, book_reflith, perplex_name=None)  
book_complith = update_mantle_density_to_reference(complith,book_complith,perplex_name=None)

if printOut:
    print("\n# Column 1: Reference MOR ########################################################################")
deltarho_ridge_correction        = 4.05  # kg/m3 # correction to reproduce the elevation difference 
                                         # in the 2-D thermo-mechanical model
geotherm_filename                = 'profile_temperature_ridge_centralvalley_GL-Tp1280_speed3_M2.tz'
density_profile_filename_ridge   = 'profile_density_ridge_centralvalley_GL-Tp1280_speed3_M2.rhoz'
path                             = './data/geodyn2d/M2/'
if printOut:
    print("Density and temperature profiles from 2-D geodynamic model M2 (temperature dependent only densities)")
lith2 = geodyn1d.lithosphere('ridge_Oc6_5_NoDiffLayer',printOut=False)
lith2.update_materials(book_reflith)
lith2.get_imposed_geotherm(path+geotherm_filename,printOut=printOut)
lith2.get_pressure_density_profile(printOut=False,
                                   filename=path+density_profile_filename_ridge,
                                       drho=deltarho_ridge_correction,
                                      zdrho=[6500,125000])



lith1 = geodyn1d.lithosphere(reflithcont,printOut=False)
lith1.update_materials(book_reflithcont)
lith1.get_steady_state_geotherm(dz=dz,printOut=True)
lith1.get_pressure_density_profile(printOut=False)
pref  = lith1.get_reference_pressure(ref_depth,printOut=True)
print('Pref = {0:7.5e} Pa with compensation depth of {1:5.1f} km'.format(pref,ref_depth/1e3))
#-------------------------------------------------------------------
lith2.set_sediment_thickness(0)
lith2.set_sea_level(sea_level_ref)
dh_ref,water_depth,sea_level,sediment_thickness = lith2.get_subsidence_with_sediments(pref,
                                                                                      ref_depth,
                                                                                      SeaLevelFixed=True,
                                                                                      printOut=False)
print('========= SUBSIDENCE AT THE REFERENCE MOR ===========')
print('WATER DEPTH        = {} m'.format(water_depth))
print('SEA LEVEL          = {} m'.format(sea_level))
print('Reference dh       = {0:6.1f} m'.format(dh_ref))

    
for deltarho_ridge_correction_MOR2 in alldeltarho_ridge_correction_MOR2:
    lith2 = geodyn1d.lithosphere('ridge_Oc6_5_NoDiffLayer',printOut=False)
    lith2.update_materials(book_complith)
    lith2.get_steady_state_geotherm(printOut=False)
    lith2.get_pressure_density_profile(printOut=False,
                                           drho=deltarho_ridge_correction_MOR2,
                                           zdrho=[6500,125000])  
    
    #-------------------------------------------------------------------
    lith2.set_sediment_thickness(0)
    lith2.set_sea_level(sea_level_ref)
    dh,water_depth,sea_level,sediment_thickness = lith2.get_subsidence_with_sediments(pref,
                                                                                      ref_depth,
                                                                                      SeaLevelFixed=True,
                                                                                      printOut=False)
               
    print('========= SUBSIDENCE AT THE RIDGE MOR2 ===========')
    print('correction mantle  = {0:6.1f} kg/m3'.format(deltarho_ridge_correction_MOR2))
    print('WATER DEPTH        = {} m'.format(water_depth))
    print('SEA LEVEL          = {} m'.format(sea_level))
    print('diff with ref      = {0:6.1f} m'.format(dh-dh_ref))


 z bottom (km) = 600000.0
 k asth. = 87.56
 Grad slm = 0.4
 Qbasal = 0.0195488281
 Tbasal = 1520.0
 Tp = 1280.0
Layer 6 T top = 1330.0 Q top = 19.55 (matSLM)
Layer 3 T top =  548.0 Q top = 19.55 (matLM1)
Layer 2 T top =  441.8 Q top = 28.61 (matLC)
Layer 1 T top =  295.3 Q top = 37.68 (matMC)
Layer 0 T top =    0.0 Q top = 51.28 (matUC)
 Qsurface = 51.28 mW/m2
Pref = 3.79540e+09 Pa with compensation depth of 125.0 km
========= SUBSIDENCE AT THE REFERENCE MOR ===========
WATER DEPTH        = 2743.614290913309 m
SEA LEVEL          = -400.0 m
Reference dh       = -3143.6 m
========= SUBSIDENCE AT THE RIDGE MOR2 ===========
correction mantle  =  -11.2 kg/m3
WATER DEPTH        = 2742.908277926956 m
SEA LEVEL          = -400.0 m
diff with ref      =    0.7 m


In [4]:
# Display temperature, pressure, and density profiles of the two columns:
#     MOR:  reference MOR column from model M2
#     MOR2: semi-analytical MOR column (same weight as MOR)

%matplotlib widget

define_rcParams()

fig=plt.figure(figsize=[10/2.54,10/2.54])
fig.subplots_adjust(hspace=0.4, wspace=0.4)
plt.plot(lith2.geotherm.T-273.15,
         lith2.geotherm.Z/1e3,
         label="MOR 2")
plt.plot(lith1.geotherm.T-273.15,lith1.geotherm.Z/1e3,
         label="Ref. lith125")
plt.xlim([0,1600])
plt.ylim([0,ref_depth/1e3+5])
plt.xlabel(r'Temperature ($^{\circ}C$)')
plt.ylabel('Depth (km)')
plt.legend()
plt.gca().invert_yaxis()

fig=plt.figure(figsize=[10/2.54,10/2.54])
plt.plot(lith2.P_rho_DepthProfile.P/1e6,
         lith2.P_rho_DepthProfile.Z/1e3,
         label="MOR 2")
plt.plot(lith1.P_rho_DepthProfile.P/1e6,lith1.P_rho_DepthProfile.Z/1e3,
         label="Ref. lith125")
plt.ylim([0,ref_depth/1e3+5])
plt.xlim([0,ref_depth*10*np.mean(lith1.P_rho_DepthProfile.rho)/1e6+200])
plt.xlabel('Pressure (MPa)')
plt.ylabel('Depth (km)')
plt.legend()
plt.gca().invert_yaxis()

fig=plt.figure(figsize=[10/2.54,10/2.54])
plt.plot(lith2.P_rho_DepthProfile.rho,
         lith2.P_rho_DepthProfile.Z/1e3,
         label="MOR 2")
plt.plot(lith1.P_rho_DepthProfile.rho,
         lith1.P_rho_DepthProfile.Z/1e3,
         label="Ref. lith125")
plt.ylim([0,ref_depth/1e3+5])
plt.xlim([2700,3600])
plt.xlabel(r'Density ($kg/m^{3}$)')
plt.ylabel('Depth (km)')
plt.legend()
plt.gca().invert_yaxis()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [5]:
# Calibration of the reference mantle density with temperature dependent only densities
#
# STEP 2: Calibration of the reference mantle density with temperature dependent only densities

water_depth_at_ridge     = 2750.0e0  # meters = reference water depth at the ridge based on average MOR elevation
sea_level_ref            = -400e0    # meters = reference sea level based on average elevation of continents
dz                       = 200e0     # meters
ref_depth                = 125e3     # meters = compensation depth (base of the continental lithosphere)

# Lithosphere 1 = Reference MOR2 column with analytical 
# The MOR from 2-D thermechanical model is used to calibrate density structure of the reference continental column 
reflith                  = 'MOR2'
book_reflith             = book_ridge
# Lithosphere 2 = Reference continental column
complith                 = 'lith125'
book_complith            = book_lith125_GL # updated parameters (./scripts/book_lith.py)


######## COMMENT/UNCOMMENT OR DEFINE A CASE #########################################################

# => Calibration of the reference mantle density
printOut              = False          # print more information
allrefdensity_mantle  = np.arange(3250,3350,10) #
refdensity_crust      = 2860
lith_depletion        = -20

# Single cases: 
#   - analytical solution for model M2
#printOut              = True          # print more information
#refdensity_crust      = 2860
#lith_depletion        = -20
#allrefdensity_mantle  = [3311]
#   - analytical solution for model SM2a (see supplemental figure S14)
#printOut              = True          # print more information
#refdensity_crust      = 2835
#lith_depletion        = 0
#allrefdensity_mantle  = [3300]

############################################################################################


book_reflith  = update_mantle_density_to_reference(reflith, book_reflith, perplex_name=None)  
book_complith = update_mantle_density_to_reference(complith,book_complith,perplex_name=None)

   
results = {
       'Reference mantle density': [],
       'Water depth': [],
       'dh': []
}

for refdensity_mantle in allrefdensity_mantle:
    
    print("Reference density mantle: {0:6.1f} kg/m3".format(refdensity_mantle))
    if printOut:
        print("\n# Column 1: Reference MOR2 ########################################################################")
        
    # Update the book-------
    book_reflith=update_book(book_reflith,'rho',refdensity_mantle,['LM1','LM2','LM3','SLM'])
    #-----------------------
    
    deltarho_ridge_correction_MOR2 = -11.25 # kg/m3 # correction to reproduce the weight of the MOR column
                                            # in the 2-D thermo-mechanical model M2 (STEP1 of the calibration)
    lith1                          = geodyn1d.lithosphere('ridge_Oc6_5_NoDiffLayer',printOut=False)
    lith1.update_materials(book_reflith)
    lith1.get_steady_state_geotherm(printOut=False)
    lith1.get_pressure_density_profile(printOut=False,
                                           drho=deltarho_ridge_correction_MOR2,
                                          zdrho=[6500,125000])    
    
    if printOut:
        print("# Column 2: Reference Continental lithosphere ###############################################################")
        
    # Update the book-------
    book_complith=update_book(book_complith,'rho',refdensity_crust,['UC','MC','LC'])
    book_complith=update_book(book_complith,'deltarho',lith_depletion,['LM1','LM2','LM3'])
    book_complith=update_book(book_complith,'rho',refdensity_mantle,['LM1','LM2','LM3','SLM'])
    #-----------------------
    
    lith2 = geodyn1d.lithosphere(complith,printOut=False)
    lith2.update_materials(book_complith)
    lith2.get_steady_state_geotherm(dz=dz,printOut=printOut)
    lith2.get_pressure_density_profile(printOut=False)
    pref  = lith2.get_reference_pressure(ref_depth,printOut=False)
    #-------------------------------------------------------------------
    lith1.set_sediment_thickness(0)
    lith1.set_sea_level(sea_level_ref)
    dh,water_depth,sea_level,sediment_thickness = lith1.get_subsidence_with_sediments(pref,
                                                                                      ref_depth,
                                                                                      SeaLevelFixed=True,
                                                                                      printOut=False)
    if printOut:
        print('========= SUBSIDENCE AT THE RIDGE WITH FIXED SEA LEVEL ===========')
        print('Reference mantle density    = {0:6.1f} kg/m3'.format(refdensity_mantle))
        print('WATER DEPTH                 = {0:6.1f} m'.format(water_depth))
        print('SEA LEVEL                   = {} m'.format(sea_level))
        print('dh                          = {0:6.1f} m'.format(dh))

    results['Reference mantle density'].append(refdensity_mantle)
    results['Water depth'].append(water_depth)
    results['dh'].append(dh)

print("Done !!")

Reference density mantle: 3250.0 kg/m3
Reference density mantle: 3260.0 kg/m3
Reference density mantle: 3270.0 kg/m3
Reference density mantle: 3280.0 kg/m3
Reference density mantle: 3290.0 kg/m3
Reference density mantle: 3300.0 kg/m3
Reference density mantle: 3310.0 kg/m3
Reference density mantle: 3320.0 kg/m3
Reference density mantle: 3330.0 kg/m3
Reference density mantle: 3340.0 kg/m3
Done !!


In [6]:
# Calibration of the reference mantle density with temperature dependent only densities
# Plot the solution from STEP 2 (Figure 10d)
#

%matplotlib widget

define_rcParams()

water_depth_at_ridge     = 2750.0e0  # meters = reference water depth at the ridge based on average MOR elevation

write_pdf     = False
x             = results['Reference mantle density']
dh            = results['dh']
waterdepthMOR = results['Water depth']

f = interp1d(waterdepthMOR,x, kind='quadratic')
xridge   = f(water_depth_at_ridge)
print("Calibration:\nThe reference mantle density is {} kg/m3 to fit the water depth at the MOR (with fixed sea level at -400 m)".format(xridge,water_depth_at_ridge))

plt.figure()
plt.plot(x,waterdepthMOR)
wdridge = np.zeros_like(x)+water_depth_at_ridge
rhoref  = np.zeros_like(waterdepthMOR)+xridge
plt.plot(x,waterdepthMOR)
plt.plot(x,wdridge)
plt.plot(rhoref,waterdepthMOR)
plt.ylabel("Water depth at the MOR (m)")
plt.xlabel(r'$\rho_{0}(mantle)$ $kg/m^{3}$')

if write_pdf:
    whatplot = 'calibration_rhoref_mantle_M2-type'
    plt.savefig("./ps/"+whatplot+".pdf",bbox_inches='tight',dpi=1200)

plt.show()

Calibration:
The reference mantle density is 3311.6524564678734 kg/m3 to fit the water depth at the MOR (with fixed sea level at -400 m)


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …